<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Prompt Engineering for Quality Decisions

Decision Intelligence applied in this module:  
* Optimizing prompts for logic, reasoning and decisions
* Examples of applying decision intelligence techniques in prompts and their output effect 
* Prompt techniques such as Chain of Thought to improve the quality of the decision reasoning and outcome 


In this module, the Semantic Kernel ability to chat completion experience will be used to optimize system prompts (personas) and instructive prompts. No new specific Semantic Kernel functionality will be introduced. This module will focus on introducing decision intelligence prompting techniques.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.22.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.22.0

Using Azure OpenAI Service


### Step 2 - Chain of Thought Reasoning for Decision Intelligence

> "The happiness of your life depends upon the quality of your thoughts." 
>
> -- <cite>Marcus Aurelius (Roman emperor, philosopher)</cite>  

One of the most effective techniques to improve logic, reasoning and overall decision-making for most LLMs is to use "Chain of Thought" (CoT) techniques. The "Chain of Thought" approach helps by breaking down complex tasks into smaller steps, making it easier to solve problems without missing important details. It improves accuracy because each step is made clear to the LLM and helps avoid mistakes. This method also makes the thought process easier to understand and explain, allowing for easier corrections when something goes wrong. The approach is useful for handling difficult concepts and multi-step problems like math or logic, making things clearer and more manageable.

The name of the "Chain of Thought" technique has been recently popularized by Generative AI. However, this technique is not new nor is it specific only to Generative AI models. Breaking down complex tasks into simpler more approachable steps and organizing thoughts has been used by decision-makers, professional services organizations and management consulting companies for quite some time. Prior to Generative AI, the concepts have been known as: "structured thinking", "MECE framework" (Mutually Exclusive, Collectively Exhaustive), and "hypothesis-driven problem-solving".  

First, approach the problem with a simple Decision prompt technique. The system decisoin prompt will be set to basic & clear decision assistant persona instructions. While this decision prompt is simple, it is still quite effective to instruct the LLM model on how to approach the decision problem. 

In [4]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var decisionResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    decisionResponse += content;
    Console.Write(content);
}

To solve this problem, the farmer needs to carefully plan the sequence of crossings to ensure that neither the goat is left alone with the wolf nor the goat is left alone with the cabbage. Here is a step-by-step plan:

1. **First Trip: Take the Goat Across**
   - The farmer takes the goat across the river and leaves it on the other side.
   - The wolf and the cabbage remain on the original side.

2. **Second Trip: Return Alone**
   - The farmer returns to the original side alone.

3. **Third Trip: Take the Wolf Across**
   - The farmer takes the wolf across the river.
   - The goat is now on the other side, and the wolf is with the farmer.

4. **Fourth Trip: Return with the Goat**
   - The farmer takes the goat back to the original side.
   - The wolf is now alone on the other side.

5. **Fifth Trip: Take the Cabbage Across**
   - The farmer takes the cabbage across the river.
   - The wolf and the cabbage are now on the other side.

6. **Sixth Trip: Return Alone**
   - The farmer retu

Approach the same problem with a much more sophisticated "Chain of Thought" (CoT) system prompt to break the problem down and think about it more in depth. Notice that the prompt below is much more detailed in how the "decision intelligence assistant" should approach the problem, highlighting things it should or should not consider and the way it should arrive at the final answer. 

Chain of Thought (CoT) prompt instructions will vary depending on how the problem needs to be approached. It can include very specific decision framework instructions, systematic decision heuristics or management consulting best practices. In general, these detailed prompts result in the LLM providing more detail and usually better outcomes. However, there are a few drawbacks to overusing Chain of Thought (CoT):
* Very long and detailed Chain of Thought (CoT) can confuse the LLM model, especially if the LLM model is smaller (i.e. GPT-4o-mini or Phi 3.5 etc.)
* GenAI LLM models hallucinate not just on output of answers, but also they can hallucinate stating that to arrive at the answer they "used" the Chain of Thought (CoT) they are describing! So, this is a very sneaky way of potentially providing a confident answer paired with a confident approach that the LLM may not even be using!

In [5]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemPromptChainOfThought = """
You are a decision intelligence assistant designed to think through problems step-by-step using Chain-of-Thought (COT) prompting. 
Before providing any answer, you must: 
Understand the Problem: Carefully read and understand the user's question or request. 
Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 
Arrive at the Final Answer: Only after completing all steps, provide the final answer or solution. 
Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Always aim to make your thought process transparent and logical. Communicate the final decision using the Minto Pyramid Principle.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPromptChainOfThought);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var decisionResponseChainOfThought = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    decisionResponseChainOfThought += content;
    Console.Write(content);
}

To solve this problem, we need to ensure that at no point are the wolf and the goat left alone together, nor the goat and the cabbage left alone together. Let's break down the steps logically:

1. **Understand the Problem:**
   - The farmer can only take one item at a time across the river.
   - The wolf cannot be left alone with the goat.
   - The goat cannot be left alone with the cabbage.

2. **Identify the Constraints:**
   - The farmer must always ensure that the goat is not left with the wolf or the cabbage when he is not present.

3. **Plan the Steps:**
   - The farmer needs to make multiple trips to ensure all items are safely transported across the river.

4. **Step-by-Step Solution:**
   - **Step 1:** The farmer takes the goat across the river first. Now the farmer is on the other side with the goat, and the wolf and cabbage are left on the original side.
   - **Step 2:** The farmer returns alone to the original side.
   - **Step 3:** The farmer takes the wolf across the rive

The Generative AI model has generated two different decision approaches to the same retirement decision question. An LLM can be used to decide which decision answer is more thought out and why. Ideally, the evaluation should be done by another family or an LLM from a different provider, not the LLM that generated the responses.  

After running the evaluation of the decision prompts below, notice the GenAI LLM model prefers the approach of the Chain of Thought (CoT) over the simple system prompt instruction.  

In [6]:
var systemPromptEvaluateResponses = """
You are an assistant that is evaluating an approach response to a question.
You will be provided with an important decision as well as the proposed approach.
Score the approach on a scale of 1 to 10 based on its comprehensiveness, clarity, and logical reasoning.
""";

var decisionEvaluationTemplateApproach1 = $"""
Decision: {puzzlePrompt}
--------------
Approach: {decisionResponse} 
""";

var decisionEvaluationTemplateApproach2 = $"""
Decision: {puzzlePrompt}
--------------
Approach: {decisionResponseChainOfThought} 
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistoryApproach1 = new ChatHistory();
chatHistoryApproach1.AddSystemMessage(systemPromptEvaluateResponses);
chatHistoryApproach1.AddUserMessage(decisionEvaluationTemplateApproach1);

var chatHistoryApproach2 = new ChatHistory();
chatHistoryApproach2.AddSystemMessage(systemPromptEvaluateResponses);
chatHistoryApproach2.AddUserMessage(decisionEvaluationTemplateApproach2);


// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

Console.WriteLine();
Console.WriteLine();
Console.WriteLine("Approach 1 Evaluation:");
Console.WriteLine("----------------------");
var evaluationResponseApproach1 = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryApproach1, openAIPromptExecutionSettings))
{
    evaluationResponseApproach1 += content;
    Console.Write(content);
}

var evaluationResponseApproach2 = string.Empty;
Console.WriteLine();
Console.WriteLine();
Console.WriteLine("Approach 2 Evaluation:");
Console.WriteLine("----------------------");
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryApproach2, openAIPromptExecutionSettings))
{
    evaluationResponseApproach2 += content;
    Console.Write(content);
}



Approach 1 Evaluation:
----------------------
Score: 9

Comprehensiveness: 9
The approach is very comprehensive, detailing each step the farmer needs to take to ensure that the goat is not eaten by the wolf and the cabbage is not eaten by the goat. It covers all necessary trips and explains the reasoning behind each move.

Clarity: 9
The steps are clearly laid out and easy to follow. Each trip is described in a straightforward manner, making it easy to understand the sequence of actions.

Logical Reasoning: 9
The logical reasoning is sound. The approach ensures that at no point are the goat and the wolf left alone together, nor are the goat and the cabbage left alone together. The sequence of trips is well thought out and effectively solves the problem.

The only minor improvement could be a brief summary at the end to reinforce the solution, but overall, the approach is excellent.

Approach 2 Evaluation:
----------------------
Score: 9

Comprehensiveness: 9
The approach is very deta

### Step 3 - Self-Consistency  

WIP 